# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats as st

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [2]:
df1 = pd.read_csv('/datasets/geo_data_0.csv')
df2 = pd.read_csv('/datasets/geo_data_1.csv')
df3 = pd.read_csv('/datasets/geo_data_2.csv')

In [3]:
display(df1.head())
display(df2.head())
display(df3.head())

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [4]:
df1.info()
df2.info()
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null 

In [5]:
print('Количеcтво дубликатов df1 по id:', df1['id'].duplicated().sum())
print('Количеcтво дубликатов df2 по id:', df2['id'].duplicated().sum())
print('Количеcтво дубликатов df3 по id:', df3['id'].duplicated().sum())

Количеcтво дубликатов df1 по id: 10
Количеcтво дубликатов df2 по id: 4
Количеcтво дубликатов df3 по id: 4


In [6]:
def drop_duplicates_by_column(df, column_name, drop=False, show_duplicates=True, show_index=False):
    duplicates_list = df[df[column_name].duplicated()][column_name].tolist()
    if show_duplicates:
        print(
            df[df['id'].isin(duplicates_list)]
            .sort_values(by='id')
        )
    duplicates_indexes = df[df['id'].isin(duplicates_list)].index
    if show_index:
        print('Duplicated indexes:', duplicates_indexes)
    if drop:
        df = df.drop(index=duplicates_indexes)
        df.reset_index(drop=True, inplace=True)
    return df

df1 = drop_duplicates_by_column(df1, 'id', True)
df2 = drop_duplicates_by_column(df2, 'id', True)
df3 = drop_duplicates_by_column(df3, 'id', True)

          id        f0        f1         f2     product
66136  74z30  1.084962 -0.312358   6.990771  127.643327
64022  74z30  0.741456  0.459229   5.153109  140.771492
51970  A5aEY -0.180335  0.935548  -2.094773   33.020205
3389   A5aEY -0.039949  0.156872   0.209861   89.249364
69163  AGS9W -0.933795  0.116194  -3.655896   19.230453
42529  AGS9W  1.454747 -0.479651   0.683380  126.370504
931    HZww2  0.755284  0.368511   1.863211   30.681774
7530   HZww2  1.061194 -0.373969  10.430210  158.828695
63593  QcMuo  0.635635 -0.473422   0.862670   64.578675
1949   QcMuo  0.506563 -0.323775  -2.215583   75.496502
75715  Tdehs  0.112079  0.430296   3.218993   60.964018
21426  Tdehs  0.829407  0.298807  -0.049563   96.035308
92341  TtcGQ  0.110711  1.022689   0.911381  101.318008
60140  TtcGQ  0.569276 -0.104876   6.440215   85.350186
89582  bsk9y  0.398908 -0.400253  10.122376  163.433078
97785  bsk9y  0.378429  0.005837   0.160827  160.637302
41724  bxg6G -0.823752  0.546319   3.630479   93

In [7]:
print(df1.shape)
print(df2.shape)
print(df3.shape)

(99980, 5)
(99992, 5)
(99992, 5)


Удалим ненужные столбцы

In [8]:
df1 = df1.drop(['id'], axis=1)
df2 = df2.drop(['id'], axis=1)
df3 = df3.drop(['id'], axis=1)

Вывод:
- Данные предобработаны;
- Сгруппировали дубликаты и отобразили их перед удалением;
- Удалили ненужные столбцы


## Обучение и проверка модели

Разделим данные

In [9]:
def prepare_Xy(df, feature_column):
    X = df.drop([feature_column], axis=1)
    y = df[feature_column]
    return X,y

X1,y1 = prepare_Xy(df1,'product')
X2,y2 = prepare_Xy(df2,'product')
X3,y3 = prepare_Xy(df3,'product')

features_train1, features_valid1, target_train1, target_valid1 = train_test_split(
    X1, y1, test_size=0.25, random_state=12345)
features_train2, features_valid2, target_train2, target_valid2 = train_test_split(
    X2, y2, test_size=0.25, random_state=12345)
features_train3, features_valid3, target_train3, target_valid3 = train_test_split(
    X3, y3, test_size=0.25, random_state=12345)

Обучение и прогнозирование на валидационных данных

In [10]:
model1 = LinearRegression(normalize = True)
model1.fit(features_train1, target_train1)
predictions_valid1 = pd.Series(model1.predict(features_valid1))

model2 = LinearRegression(normalize = True)
model2.fit(features_train2, target_train2)
predictions_valid2 = pd.Series(model2.predict(features_valid2))

model3 = LinearRegression(normalize = True)
model3.fit(features_train3, target_train3)
predictions_valid3 = pd.Series(model3.predict(features_valid3))


target_valid_i1 = target_valid1.reset_index(drop=True)
target_valid_i2 = target_valid2.reset_index(drop=True)
target_valid_i3 = target_valid3.reset_index(drop=True)

In [11]:
model1.get_params(deep=True)

{'copy_X': True,
 'fit_intercept': True,
 'n_jobs': None,
 'normalize': True,
 'positive': False}

In [12]:
result1 = mean_squared_error(target_valid_i1, 
                            predictions_valid1, 
                            squared=False)

result2 = mean_squared_error(target_valid_i2, 
                            predictions_valid2, 
                            squared=False)

result3 = mean_squared_error(target_valid_i3, 
                            predictions_valid3, 
                            squared=False)
print('Регион 1.')
print('Средний запас предсказанного сырья:', predictions_valid1.mean())
print('RMSE модели линейной регрессии на валидационной выборке:', result1, '\n')

print('Регион 2.')
print('Средний запас предсказанного сырья:', predictions_valid2.mean())
print('RMSE модели линейной регрессии на валидационной выборке:', result2, '\n')

print('Регион 3.')
print('Средний запас предсказанного сырья:', predictions_valid3.mean())
print('RMSE модели линейной регрессии на валидационной выборке:', result3)

Регион 1.
Средний запас предсказанного сырья: 92.42384109947359
RMSE модели линейной регрессии на валидационной выборке: 37.716904960382735 

Регион 2.
Средний запас предсказанного сырья: 68.98311857983123
RMSE модели линейной регрессии на валидационной выборке: 0.8914901390348537 

Регион 3.
Средний запас предсказанного сырья: 95.11622302076479
RMSE модели линейной регрессии на валидационной выборке: 39.975543264382345


Средний запас сырья самый низкий в регионе 2, но RMSE там лучший. Район 1 и 3 выглядит адекватно, как и значение RMSE.

## Подготовка к расчёту прибыли

In [13]:
BUDGET = 10 * 10 ** 9
INCOME_1K = 450000
LOSS_PROB = 0.025

POINTS_TO_EXPLORE = 500
BEST_POINTS = 200

In [14]:
sufficient_volume = BUDGET/INCOME_1K
print('Объём безубыточности разработки, тыс. баррелей: %.2f' % sufficient_volume)
sufficient_volume_per_well = sufficient_volume/BEST_POINTS
print('Объём безубыточности разработки, для одной скважины %.2f' % sufficient_volume_per_well)

print('Регион 1 средний запас в скважине, тыс. баррелей: %.2f' % df1['product'].mean())
print('Регион 2 средний запас в скважине, тыс. баррелей: %.2f' % df2['product'].mean())
print('Регион 3 средний запас в скважине, тыс. баррелей: %.2f' % df3['product'].mean())

Объём безубыточности разработки, тыс. баррелей: 22222.22
Объём безубыточности разработки, для одной скважины 111.11
Регион 1 средний запас в скважине, тыс. баррелей: 92.50
Регион 2 средний запас в скважине, тыс. баррелей: 68.82
Регион 3 средний запас в скважине, тыс. баррелей: 95.00


Получается объем безубыточной разработки больше, чем средние запасы по скважине в каждом из регионов. Это значит, что случайный выбор скважин не даст прибыли и мы пойдем в убыток.

## Расчёт прибыли и рисков 

функция для расчета прибыли от выбранных скважин и прогнозов модели

In [15]:
def max_profit(predictions, target, sufficient_volume):
    predictions_sorted_best = predictions.sort_values(ascending=False)
    predicted_volume_total = target[predictions_sorted_best.index][:BEST_POINTS].sum()
    profit = (predicted_volume_total - sufficient_volume) * INCOME_1K
    return profit

Bootstrap с 1000 выборками для определения распределения прибыли

In [16]:
def bootstrap_4region(predictions_valid, target_valid_i):
    
    state = np.random.RandomState(12345)
    values = []
    
    for i in range(1000):
        predictions_subsample = predictions_valid.sample(n=500, replace=True, random_state=state)
        values.append(max_profit(predictions_subsample, target_valid_i, sufficient_volume)) 

    values = pd.Series(values)
    return values

In [17]:
val = []
val.append(bootstrap_4region(predictions_valid1, target_valid_i1))
val.append(bootstrap_4region(predictions_valid2, target_valid_i2))
val.append(bootstrap_4region(predictions_valid3, target_valid_i3))

Находим среднюю доходность, 95% доверительный интервал и риск потери. Убыток - это отрицательная прибыль.

In [18]:
for i in range(3):
    lower = val[i].quantile(0.025)
    higher = val[i].quantile(0.975)
    print(f'Регион {i+1} \n  Процент прибыльных в 1000 выборок: {(val[i] > 0).mean()*100}%')
    print(f'  Вероятность убытка: {(val[i] < 0).mean()*100}%')
    print(f'  Среднее значение прибыли: {int(val[i].mean()):,}')
    print(f'  95%-й доверительный интервал:')
    print(f'     Нижний квантиль {lower:,.0f}')
    print(f'     Верхний квантиль {higher:,.0f}')

Регион 1 
  Процент прибыльных в 1000 выборок: 94.5%
  Вероятность убытка: 5.5%
  Среднее значение прибыли: 431,538,186
  95%-й доверительный интервал:
     Нижний квантиль -80,924,627
     Верхний квантиль 941,037,638
Регион 2 
  Процент прибыльных в 1000 выборок: 98.0%
  Вероятность убытка: 2.0%
  Среднее значение прибыли: 477,948,839
  95%-й доверительный интервал:
     Нижний квантиль 51,741,836
     Верхний квантиль 897,944,131
Регион 3 
  Процент прибыльных в 1000 выборок: 87.7%
  Вероятность убытка: 12.3%
  Среднее значение прибыли: 322,147,249
  95%-й доверительный интервал:
     Нижний квантиль -173,464,434
     Верхний квантиль 843,529,290


Рентабельность более 95% наблюдается только в регионе 2.

Регион 1 не оправдал ожиданий.

В этом случае можно рекомендовать рассматривать для производства только второй регион.
Впрочем, и первый регион тоже достоин права считаться при условии увеличения риска на 0,5%.